In [1]:
import pandas as pd

In [2]:
!nvidia-smi


Mon Dec 22 21:33:39 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   28C    P0             47W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
!pip -q install -U nnunetv2 nibabel tifffile simpleitk blosc2 kaggle matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.4/86.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25

Cell 3 — Set up Kaggle authentication

Why: Kaggle downloads require kaggle.json API key.

What you must do:

Kaggle → Account → Create New API Token

It downloads kaggle.json

Upload kaggle.json into Colab (Files panel)

In [5]:
import os, pathlib, stat

kaggle_json = "/content/kaggle.json"
assert os.path.exists(kaggle_json), "Upload kaggle.json to /content/ first (Files panel)."

pathlib.Path("/root/.kaggle").mkdir(parents=True, exist_ok=True)
!cp /content/kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

print("Kaggle configured.")


Kaggle configured.


Download dataset from Kaggle + unzip

Why: brings the data into Colab storage.

In [8]:
import os

KAGGLE_DATASET = "murillobouzon/kssd2025-kidney-stone-segmentation-dataset"  # <-- CHANGE THIS
OUT_DIR = "/content/KSSD2025_kaggle"
os.makedirs(OUT_DIR, exist_ok=True)

!kaggle datasets download -d {KAGGLE_DATASET} -p {OUT_DIR} --unzip

print("Downloaded and unzipped to:", OUT_DIR)
!find {OUT_DIR} -maxdepth 3 -type d | head -n 50

Dataset URL: https://www.kaggle.com/datasets/murillobouzon/kssd2025-kidney-stone-segmentation-dataset
License(s): unknown
 93% 73.0M/78.5M [00:00<00:00, 143MB/s]
100% 78.5M/78.5M [00:00<00:00, 146MB/s]
Downloaded and unzipped to: /content/KSSD2025_kaggle
/content/KSSD2025_kaggle
/content/KSSD2025_kaggle/data
/content/KSSD2025_kaggle/data/label
/content/KSSD2025_kaggle/data/image


Locate images/labels folders

Why: Kaggle zips sometimes have different folder names. We auto-detect.

In [9]:
import os, glob

def find_folder(root, candidates):
    for c in candidates:
        p = os.path.join(root, c)
        if os.path.isdir(p):
            return p
    return None

root = OUT_DIR

# Common patterns (edit if your Kaggle zip uses different names)
image_candidates = [
    "dataset/image", "dataset/images", "images", "image", "data/image", "data/images"
]
label_candidates = [
    "dataset/label", "dataset/labels", "labels", "label", "data/label", "data/labels", "mask", "masks"
]

img_dir = find_folder(root, image_candidates)
lbl_dir = find_folder(root, label_candidates)

# If not found, try recursive search for folders containing .tif
if img_dir is None:
    tif_dirs = {}
    for p in glob.glob(root + "/**/*.tif", recursive=True):
        d = os.path.dirname(p)
        tif_dirs[d] = tif_dirs.get(d, 0) + 1
    # pick top tif folder as image guess
    if tif_dirs:
        img_dir = sorted(tif_dirs.items(), key=lambda x: -x[1])[0][0]

print("img_dir:", img_dir)
print("lbl_dir:", lbl_dir)

# sanity list
if img_dir:
    print("Sample images:", sorted(os.listdir(img_dir))[:10])
if lbl_dir:
    print("Sample labels:", sorted(os.listdir(lbl_dir))[:10])


img_dir: /content/KSSD2025_kaggle/data/image
lbl_dir: /content/KSSD2025_kaggle/data/label
Sample images: ['1.tif', '10.tif', '1000.tif', '1001.tif', '1002.tif', '1003.tif', '1012.tif', '1013.tif', '1014.tif', '1015.tif']
Sample labels: ['1.tif', '10.tif', '1000.tif', '1001.tif', '1002.tif', '1003.tif', '1012.tif', '1013.tif', '1014.tif', '1015.tif']


Set nnU-Net directories

In [10]:
import os

BASE = "/content/nnUNet"
os.environ["nnUNet_raw"] = f"{BASE}/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = f"{BASE}/nnUNet_preprocessed"
os.environ["nnUNet_results"] = f"{BASE}/nnUNet_results"

for p in [os.environ["nnUNet_raw"], os.environ["nnUNet_preprocessed"], os.environ["nnUNet_results"]]:
    os.makedirs(p, exist_ok=True)

print("nnUNet_raw:", os.environ["nnUNet_raw"])
print("nnUNet_preprocessed:", os.environ["nnUNet_preprocessed"])
print("nnUNet_results:", os.environ["nnUNet_results"])


nnUNet_raw: /content/nnUNet/nnUNet_raw
nnUNet_preprocessed: /content/nnUNet/nnUNet_preprocessed
nnUNet_results: /content/nnUNet/nnUNet_results


Create nnU-Net dataset folders

In [11]:
import os

DATASET_ID = 501
DATASET_NAME = "KSSD"

nn_folder = f"{os.environ['nnUNet_raw']}/Dataset{DATASET_ID:03d}_{DATASET_NAME}"
imagesTr = f"{nn_folder}/imagesTr"
labelsTr = f"{nn_folder}/labelsTr"

os.makedirs(imagesTr, exist_ok=True)
os.makedirs(labelsTr, exist_ok=True)

print("nn_folder:", nn_folder)


nn_folder: /content/nnUNet/nnUNet_raw/Dataset501_KSSD


convert .tif → .nii.gz with label remap

In [12]:
import glob
import numpy as np
import tifffile as tiff
import nibabel as nib

img_files = sorted(glob.glob(os.path.join(img_dir, "*.tif")))
lbl_files = sorted(glob.glob(os.path.join(lbl_dir, "*.tif")))

print("Found images:", len(img_files))
print("Found labels:", len(lbl_files))
assert len(img_files) == len(lbl_files), "Mismatch counts. Check img_dir/lbl_dir."

for i, (img_p, lbl_p) in enumerate(zip(img_files, lbl_files)):
    img = tiff.imread(img_p).astype(np.float32)
    lbl = tiff.imread(lbl_p).astype(np.uint8)

    # IMPORTANT: binary remap based on your earlier discovery
    lbl = (lbl >= 251).astype(np.uint8)

    if img.ndim == 2:
        img = img[..., None]
    if lbl.ndim == 2:
        lbl = lbl[..., None]

    affine = np.eye(4)
    case_id = f"case_{i:04d}"

    nib.save(nib.Nifti1Image(img, affine), f"{imagesTr}/{case_id}_0000.nii.gz")
    nib.save(nib.Nifti1Image(lbl, affine), f"{labelsTr}/{case_id}.nii.gz")

print("Conversion finished.")


Found images: 838
Found labels: 838
Conversion finished.


Confirm labels are clean {0,1}

In [13]:
import glob, numpy as np
import nibabel as nib

sample = sorted(glob.glob(labelsTr + "/*.nii.gz"))[0]
u = np.unique(nib.load(sample).get_fdata())
print("Unique label values in converted label:", u)


Unique label values in converted label: [0. 1.]


Write dataset.json

Why: nnU-Net needs class names and file ending.

In [14]:
import json, glob

dataset_json = {
    "channel_names": {"0": "CT"},
    "labels": {"background": 0, "stone": 1},
    "numTraining": len(glob.glob(imagesTr + "/*.nii.gz")),
    "file_ending": ".nii.gz"
}

with open(f"{nn_folder}/dataset.json", "w") as f:
    json.dump(dataset_json, f, indent=2)

print("Wrote:", f"{nn_folder}/dataset.json")
print(dataset_json)


Wrote: /content/nnUNet/nnUNet_raw/Dataset501_KSSD/dataset.json
{'channel_names': {'0': 'CT'}, 'labels': {'background': 0, 'stone': 1}, 'numTraining': 838, 'file_ending': '.nii.gz'}


Plan + preprocess

Why: nnU-Net auto-configures everything.

In [15]:
!nnUNetv2_plan_and_preprocess -d 501 --verify_dataset_integrity


Fingerprint extraction...
Dataset501_KSSD
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> as reader/writer
100% 838/838 [00:07<00:00, 109.23it/s]
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 14, 'patch_size': (np.int64(512), np.int64(448)), 'median_image_size_in_voxels': array([512., 416.]), 'spacing': array([1., 1.]), 'normalization_schemes': ['CTNormalization

In [16]:
!nnUNetv2_train 501 2d 0


Streaming output truncated to the last 5000 lines.
2025-12-23 00:02:37.455714: val_loss -0.9842
2025-12-23 00:02:37.455827: Pseudo dice [np.float32(0.9882)]
2025-12-23 00:02:37.455936: Epoch time: 22.41 s
2025-12-23 00:02:38.898661: 
2025-12-23 00:02:38.898940: Epoch 344
2025-12-23 00:02:38.899075: Current learning rate: 0.00684
2025-12-23 00:03:01.339194: train_loss -0.9667
2025-12-23 00:03:01.339449: val_loss -0.9781
2025-12-23 00:03:01.339546: Pseudo dice [np.float32(0.984)]
2025-12-23 00:03:01.339637: Epoch time: 22.44 s
2025-12-23 00:03:02.768365: 
2025-12-23 00:03:02.768631: Epoch 345
2025-12-23 00:03:02.768795: Current learning rate: 0.00683
2025-12-23 00:03:25.214919: train_loss -0.9656
2025-12-23 00:03:25.215166: val_loss -0.9825
2025-12-23 00:03:25.215299: Pseudo dice [np.float32(0.987)]
2025-12-23 00:03:25.215400: Epoch time: 22.45 s
2025-12-23 00:03:26.587313: 
2025-12-23 00:03:26.587517: Epoch 346
2025-12-23 00:03:26.587672: Current learning rate: 0.00682
2025-12-23 00:03:

In [17]:
ls $nnUNet_results/Dataset501*/nnUNetTrainerV2__nnUNetPlans__2d/fold_0


ls: cannot access '/content/nnUNet/nnUNet_results/Dataset501*/nnUNetTrainerV2__nnUNetPlans__2d/fold_0': No such file or directory


In [19]:
!echo $nnUNet_results

/content/nnUNet/nnUNet_results


In [20]:
ls $nnUNet_results/Dataset501_KSSD2025


ls: cannot access '/content/nnUNet/nnUNet_results/Dataset501_KSSD2025': No such file or directory


In [21]:
ls -lah /content/nnUNet/nnUNet_results


total 12K
drwxr-xr-x 3 root root 4.0K Dec 22 21:42 ./
drwxr-xr-x 5 root root 4.0K Dec 22 21:38 ../
drwxr-xr-x 3 root root 4.0K Dec 22 21:42 Dataset501_KSSD/


In [22]:
ls -lah /content/nnUNet/nnUNet_results/Dataset501*


total 12K
drwxr-xr-x 3 root root 4.0K Dec 22 21:42 ./
drwxr-xr-x 3 root root 4.0K Dec 22 21:42 ../
drwxr-xr-x 3 root root 4.0K Dec 22 21:42 nnUNetTrainer__nnUNetPlans__2d/


In [24]:
!find /content/nnUNet/nnUNet_results/Dataset501* -maxdepth 2 -type d

/content/nnUNet/nnUNet_results/Dataset501_KSSD
/content/nnUNet/nnUNet_results/Dataset501_KSSD/nnUNetTrainer__nnUNetPlans__2d
/content/nnUNet/nnUNet_results/Dataset501_KSSD/nnUNetTrainer__nnUNetPlans__2d/fold_0


In [25]:
!ls -lah /content/nnUNet/nnUNet_results/Dataset501_KSSD/nnUNetTrainer__nnUNetPlans__2d/fold_0


total 513M
drwxr-xr-x 3 root root 4.0K Dec 23 04:24 .
drwxr-xr-x 3 root root 4.0K Dec 22 21:42 ..
-rw-r--r-- 1 root root 256M Dec 23 04:09 checkpoint_best.pth
-rw-r--r-- 1 root root 256M Dec 23 04:24 checkpoint_final.pth
-rw-r--r-- 1 root root 9.1K Dec 22 21:42 debug.json
-rw-r--r-- 1 root root 422K Dec 23 04:24 progress.png
-rw-r--r-- 1 root root 366K Dec 23 04:24 training_log_2025_12_22_21_42_26.txt
drwxr-xr-x 2 root root 4.0K Dec 23 04:24 validation


In [28]:
import json, os, glob

base_dir = "/content/nnUNet/nnUNet_results/Dataset501_KSSD/nnUNetTrainer__nnUNetPlans__2d/fold_0/"
log_files = glob.glob(os.path.join(base_dir, "training_log*.json")) + \
            glob.glob(os.path.join(base_dir, "training_log*.txt"))

if not log_files:
    print("Error: No training log file found in the specified directory.")
    best = None
else:
    # Sort by modification time to get the latest file, assuming newer is better
    log_files.sort(key=os.path.getmtime, reverse=True)
    log_path = log_files[0]
    print(f"Found log file: {log_path}")

    best_dice = None
    try:
        # Attempt to load as JSON (for future compatibility or if nnU-Net changes output format)
        with open(log_path, "r") as f:
            log = json.load(f)

        # Original logic for JSON (if 'epochs' key exists)
        if "epochs" in log:
            candidates = []
            for e in log["epochs"]:
                for path in [
                    ("validation", "Dice"), ("validation", "mean_dice"),
                    ("validation", "foreground_dice"), ("val", "Dice"),
                    ("val", "mean_dice"),
                ]:
                    a = e
                    ok = True
                    for k in path:
                        if isinstance(a, dict) and k in a:
                            a = a[k]
                        else:
                            ok = False
                            break
                    if ok and isinstance(a, (int, float)):
                        candidates.append(a)
            if candidates:
                best_dice = max(candidates)

    except json.JSONDecodeError:
        # If not JSON, try parsing as text
        print("Log file is not JSON, attempting to parse as plain text...")
        with open(log_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if "Yayy! New best EMA pseudo Dice:" in line:
                    try:
                        best_dice = float(line.split(':')[1].strip())
                    except (ValueError, IndexError):
                        pass # Continue if parsing fails for a specific line

    best = best_dice

print("Best validation Dice (fold 0):", best)


Found log file: /content/nnUNet/nnUNet_results/Dataset501_KSSD/nnUNetTrainer__nnUNetPlans__2d/fold_0/training_log_2025_12_22_21_42_26.txt
Log file is not JSON, attempting to parse as plain text...
Best validation Dice (fold 0): 9.0


In [29]:
!ls -lah /content/nnUNet/nnUNet_results/Dataset501_KSSD/nnUNetTrainer__nnUNetPlans__2d/fold_0 | grep -i log


-rw-r--r-- 1 root root 366K Dec 23 04:24 training_log_2025_12_22_21_42_26.txt


In [30]:
!find /content/nnUNet/nnUNet_results/Dataset501_KSSD/nnUNetTrainer__nnUNetPlans__2d/fold_0 -maxdepth 2 -type f -name "*.json" -print


/content/nnUNet/nnUNet_results/Dataset501_KSSD/nnUNetTrainer__nnUNetPlans__2d/fold_0/validation/summary.json
/content/nnUNet/nnUNet_results/Dataset501_KSSD/nnUNetTrainer__nnUNetPlans__2d/fold_0/debug.json


In [31]:
import json

p = "/content/nnUNet/nnUNet_results/Dataset501_KSSD/nnUNetTrainer__nnUNetPlans__2d/fold_0/validation/summary.json"
with open(p, "r") as f:
    s = json.load(f)

print(json.dumps(s, indent=2)[:4000])  # print first part nicely


{
  "foreground_mean": {
    "Dice": 0.98538243198826,
    "FN": 1.244047619047619,
    "FP": 1.8571428571428572,
    "IoU": 0.9719722515597956,
    "TN": 261976.04166666666,
    "TP": 164.85714285714286,
    "n_pred": 166.71428571428572,
    "n_ref": 166.10119047619048
  },
  "mean": {
    "1": {
      "Dice": 0.98538243198826,
      "FN": 1.244047619047619,
      "FP": 1.8571428571428572,
      "IoU": 0.9719722515597956,
      "TN": 261976.04166666666,
      "TP": 164.85714285714286,
      "n_pred": 166.71428571428572,
      "n_ref": 166.10119047619048
    }
  },
  "metric_per_case": [
    {
      "metrics": {
        "1": {
          "Dice": 0.9901960784313726,
          "FN": 0,
          "FP": 2,
          "IoU": 0.9805825242718447,
          "TN": 262041,
          "TP": 101,
          "n_pred": 103,
          "n_ref": 101
        }
      },
      "prediction_file": "/content/nnUNet/nnUNet_results/Dataset501_KSSD/nnUNetTrainer__nnUNetPlans__2d/fold_0/validation/case_0000.nii.gz",

In [32]:
import json

p = "/content/nnUNet/nnUNet_results/Dataset501_KSSD/nnUNetTrainer__nnUNetPlans__2d/fold_0/validation/summary.json"
with open(p, "r") as f:
    s = json.load(f)

def find_dice(obj, path=""):
    hits = []
    if isinstance(obj, dict):
        for k, v in obj.items():
            newp = f"{path}.{k}" if path else k
            if isinstance(v, (int, float)) and ("dice" in k.lower() or "dsc" in k.lower()):
                hits.append((newp, v))
            else:
                hits += find_dice(v, newp)
    elif isinstance(obj, list):
        for i, v in enumerate(obj):
            hits += find_dice(v, f"{path}[{i}]")
    return hits

hits = find_dice(s)
print("Dice-like fields found:")
for k, v in hits:
    print(k, "=", v)


Dice-like fields found:
foreground_mean.Dice = 0.98538243198826
mean.1.Dice = 0.98538243198826
metric_per_case[0].metrics.1.Dice = 0.9901960784313726
metric_per_case[1].metrics.1.Dice = 0.958904109589041
metric_per_case[2].metrics.1.Dice = 1.0
metric_per_case[3].metrics.1.Dice = 0.9333333333333333
metric_per_case[4].metrics.1.Dice = 0.9585798816568047
metric_per_case[5].metrics.1.Dice = 0.9771689497716894
metric_per_case[6].metrics.1.Dice = 0.9917898193760263
metric_per_case[7].metrics.1.Dice = 0.9896907216494846
metric_per_case[8].metrics.1.Dice = 0.9968152866242038
metric_per_case[9].metrics.1.Dice = 0.9758620689655172
metric_per_case[10].metrics.1.Dice = 0.9818956336528222
metric_per_case[11].metrics.1.Dice = 0.9898785425101214
metric_per_case[12].metrics.1.Dice = 0.9812382739212008
metric_per_case[13].metrics.1.Dice = 0.9923857868020305
metric_per_case[14].metrics.1.Dice = 0.983225806451613
metric_per_case[15].metrics.1.Dice = 0.9807280513918629
metric_per_case[16].metrics.1.Dice =

Train Fold 1

In [35]:
!nnUNetv2_train 501 2d 1 --device cuda


usage: nnUNetv2_train [-h] [-tr TR] [-p P]
                      [-pretrained_weights PRETRAINED_WEIGHTS]
                      [-num_gpus NUM_GPUS] [--npz] [--c] [--val] [--val_best]
                      [--disable_checkpointing] [-device DEVICE]
                      dataset_name_or_id configuration fold
nnUNetv2_train: error: unrecognized arguments: --device cuda


In [ ]:
!nnUNetv2_train 501 2d 1



############################
INFO: You are using the old nnU-Net default plans. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Using device: cuda:0

#######################################################################
Please cite the following paper when using nnU-Net:
Isensee, F., Jaeger, P. F., Kohl, S. A., Petersen, J., & Maier-Hein, K. H. (2021). nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation. Nature methods, 18(2), 203-211.
#######################################################################

2025-12-23 04:55:12.660636: Using torch.compile...
2025-12-23 04:55:14.191850: do_dummy_2d_data_aug: False
2025-12-23 04:55:14.194009: Using splits from existing split file: /content/nnUNet/nnUNet_preprocessed/Dataset501_KSSD/splits_final.json
2025-12-23 04:55:14.194555: The split file contains 